In [2]:
#All the imports here
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.visualization.process_tree import visualizer as pt_visualizer
from pm4py.objects.conversion.process_tree import converter as pt_converter
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.visualization.dfg import visualizer as dfg_visualization
from pm4py.objects.conversion.dfg import converter as dfg_mining
from pm4py.visualization.petrinet import factory as pn_vis_factory
from collections import defaultdict 
import pandas as pd
from pm4py.objects.log.util import dataframe_utils
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.adapters.pandas import csv_import_adapter
from pm4py.objects.conversion.log import factory as conversion_factory
from pm4py.util import constants
import math
from datetime import date
import numpy as np
from pm4py.objects.log.util import sorting
from pm4py.algo.filtering.log.attributes import attributes_filter
    

In [33]:
#####   All auxilary functions   ########
#Checks for the largest common prefix  
def lcp(s, t):  
  n = min(len(s),len(t));  
  for i in range(0,n):  
    if(s[i] != t[i]):  
      return s[0:i];  
  else:  
    return s[0:n];  

def Find_sequence(eventList):
    lrs="";  
    n = len(eventList);  
    for i in range(0,n):  
      for j in range(i+1,n):  
        #Checks for the largest common factors in every substring  
        x = lcp(eventList[i:n],eventList[j:n]);  
            #If the current prefix is greater than previous one   
            #then it takes the current one as longest repeating sequence  
        if(len(x) > len(lrs)):
            lrs=x;
    
          
            
    if(len(set(lrs))>1):
        return (lrs);  

def find_duplicate_events(x): 
    _size = len(x) 
    duplicate_list = [] 
    for i in range(_size): 
        k = i + 1
        for j in range(k, _size): 
            if x[i] == x[j] and x[i] not in duplicate_list: 
                duplicate_list.append(x[i]) 
    return duplicate_list

In [52]:
def DetectWeaknesses(logSorted, blacklist, maxTime, df, cols):
    
    #print("In detectWeaknesses function",df)
    for case_index, case in enumerate(logSorted):
        eventList=[]
        lrs=""
        indexList=[]
        d={}
        l=""
        prev=""
        d2={}
        l2=[]
        prev_end_timestamp=0
        idle_time=0
        prev_activity=""
        duration=0 
        a=""
        max_duration=0
        for event_index, event in enumerate(case):
            eventList.append(event["Activity"])  
            if(event["Activity"] in blacklist):
                #print("Unwanted activity=> activity: %s -> case: %s that started @ %s " % (event["Activity"], event["Case ID"], event["Start Timestamp"]))
                row={cols[0]:event["Case ID"]+"-> Event "+str(event_index), cols[1]: event["Case ID"], cols[2]:'AF', cols[3]:'1',cols[4]:'Expert',cols[5]:event["Start Timestamp"],cols[6]:'Unwanted activity \"'+event["Activity"]+'\"',cols[7]:'In the case', cols[8]:'Event level'}
                df=df.append(row, ignore_index=True)
                
            if( prev!="" and  event["Resource"]!= prev):
                #print("The resource has changed for the activity: \"%s\" from \"%s\" to \"%s\""%(event["Activity"], prev, event["Resource"]))
                row={cols[0]:event["Case ID"]+"-> Event "+str(event_index), cols[1]: event["Case ID"], cols[2]:'AF', cols[3]:'4',cols[4]:'Automatic detection',cols[5]:event["Start Timestamp"],cols[6]:'Change of interface for activity '+event["Activity"]+' from ' +prev+' to '+ event["Resource"],cols[7]:'In the case',cols[8]:'Event Level'}
                df=df.append(row, ignore_index=True)
            prev=event["Resource"]
            
            if(prev_end_timestamp!=0):
                idle_time=pd.to_datetime(event["Start Timestamp"], format = "%m/%d/%Y %H:%M:%S")-prev_end_timestamp
            if(type(idle_time)!= int and idle_time.total_seconds()>maxTime):
                #.total_seconds()>7200) :
                row={cols[0]:event["Case ID"]+"-> Event "+str(event_index), cols[1]: event["Case ID"], cols[2]:'PA', cols[3]:'6',cols[4]:'Expert',cols[5]:event["Start Timestamp"],cols[6]:'Idletime between '+prev_activity+' to ' +event["Activity"]+' is '+ str(idle_time),cols[7]:'In the case',cols[8]:'Event level'}
                df=df.append(row, ignore_index=True)
            prev_end_timestamp=pd.to_datetime(event["Complete Timestamp"], format = "%m/%d/%Y %H:%M:%S") 
            prev_activity=event["Activity"]
            
            if event["Activity"] not in d2.keys():
                l2=[]
            else:
                l2=d2[event["Activity"]]
            l2.append( (pd.to_datetime(event["Complete Timestamp"], format = "%m/%d/%Y %H:%M:%S")-pd.to_datetime(event["Start Timestamp"], format = "%m/%d/%Y %H:%M:%S"))/ pd.Timedelta(hours=1))
            d2[event["Activity"]]=(l2)
            
            duration=pd.to_datetime(event["Complete Timestamp"], format = "%m/%d/%Y %H:%M:%S")-pd.to_datetime(event["Start Timestamp"], format = "%m/%d/%Y %H:%M:%S")
            if(max_duration==0 or duration>max_duration):
                max_duration=duration
                a=event["Activity"]
    
    
    
                
        if(Find_sequence(eventList) is not None ):
            lrs=Find_sequence(eventList)
            #print("Repeating sequence for events in case:",case.attributes['concept:name']," is: ", lrs)  
            row={cols[0]:case.attributes['concept:name'], cols[1]: case.attributes['concept:name'], cols[2]:'AF', cols[3]:'2',cols[4]:'Automatic detection',cols[5]:'',cols[6]:'Backloop {'+''.join(lrs)+'}',cols[7]:'In the case',cols[8]:'Case level'}
            df=df.append(row, ignore_index=True)
            
        duplicateEventList=[]
        duplicateEventList=find_duplicate_events(eventList)
        #print ("The events which got repeated in the trace are",duplicateEventList)
        if(len(duplicateEventList)>0):
            row={cols[0]:case.attributes["concept:name"], cols[1]: event["Case ID"], cols[2]:'AF', cols[3]:'3',cols[4]:'Automatic detection',cols[5]:"",cols[6]:'Redundant Activities list: \"'+''.join(duplicateEventList)+'\"',cols[7]:'In the case', cols[8]:'Case Level'}
            df=df.append(row, ignore_index=True)
            
    
    row={cols[0]:"All Activities", cols[1]: a, cols[2]:'PA', cols[3]:'8',cols[4]:'Automatic detection',cols[5]:'',cols[6]:'Activity took maximum time of '+str(max_duration),cols[7]:'In the Activity',cols[8]:'Log Level'}
    df=df.append(row, ignore_index=True)
    
    variance_dict={}
    for k,v in d2.items():
        variance_dict[k]= (min(v), max(v),np.mean(v) ,np.var(v))
        row={cols[0]:"All Activities", cols[1]: k, cols[2]:'PA', cols[3]:'7',cols[4]:'Automatic detection',cols[5]:'',cols[6]:'(Min, Max, Average, Variance) for current activity:'+str((min(v), max(v),np.mean(v) ,np.var(v))),cols[7]:'In the Activity',cols[8]:'Activity Level'}
        df=df.append(row, ignore_index=True)
        
    return df
    

In [53]:
def Parallelizable_tasks_CaseLevel(df,cols):

    print("Parallelizable_tasks_CaseLevel function\n\n")
    log_csv3 = pd.read_csv('Production_Data.csv', sep=',')
    log_csv3.rename(columns={'Activity': 'concept:name'}, inplace=True)
    parameters = {log_converter.Variants.TO_EVENT_LOG.value.Parameters.CASE_ID_KEY: 'Case ID'}
    event_log3 = log_converter.apply(log_csv3, parameters=parameters, variant=log_converter.Variants.TO_EVENT_LOG)
    event_log3 = sorting.sort_timestamp(event_log3,"Start Timestamp", False)
    
    for case_index, case in enumerate(event_log3):
        
        tracefilter_log_pos = attributes_filter.apply(event_log3, [case.attributes["concept:name"]],
                                          parameters={attributes_filter.Parameters.ATTRIBUTE_KEY : "Case ID", attributes_filter.Parameters.POSITIVE: True})
        
        dfg_simple3 = dfg_discovery.apply(tracefilter_log_pos)
        l=[]
        for k in dfg_simple3.keys():
            if(k[0]!=k[1]):
                if (k[1],k[0]) in dfg_simple3.keys():
                    l.append((k[1],k[0]))
        l1=[]
        for i in l:
            if (i[1],i[0]) in l:
                l1.append((i[0],i[1]))
                l.remove((i[1],i[0]))
                l.remove((i[0],i[1]))
        if(len(l)>0)   :  
            row={cols[0]:case.attributes['concept:name'], cols[1]: case.attributes['concept:name'], cols[2]:'AF', cols[3]:'9',cols[4]:'Automatic detection',cols[5]:'',cols[6]:'Parallelizable tasks :'+''.join(str(l1)),cols[7]:'In the case',cols[8]:'Case level'}
            df=df.append(row, ignore_index=True)
            #print("\n\nParallelizable tasks for Case:",case.attributes["concept:name"]," are => ", end=" ")
            #print(l1)
        
    return df
            

In [54]:
def main(): 
    print("Welcome to Joint Master thesis:\nModelling of production expertise to extend the data-driven analysis of process models") 
    ######All initialisations############
    blacklist=[ 'Lapping - Machine 1','Turning & Milling - Machine 8']
    cols=['Detected Weakness Row','Case ID','Weakness Type (AF/PA)','Weakness ID','Weakness Origin', 'Weakness Time','Weakness Information','Weakness Measurement', 'Weakness Level']
    df=pd.DataFrame(columns=cols)
    
    
    #log = xes_importer.apply('running-example.xes')
    log_csv = pd.read_csv('Production_Data.csv', sep=',')
    log = conversion_factory.apply(log_csv, parameters={constants.PARAMETER_CONSTANT_CASEID_KEY: "Case ID",
                                                   constants.PARAMETER_CONSTANT_ACTIVITY_KEY: "Activity",
                                                    constants.PARAMETER_CONSTANT_START_TIMESTAMP_KEY:"Start Timestamp",
                                                    constants.PARAMETER_CONSTANT_RESOURCE_KEY:"Resource",
                                                    constants.PARAMETER_CONSTANT_TIMESTAMP_KEY:"Complete Timestamp"
                                                   })

    
    maxTime=86400
    logSorted = sorting.sort_timestamp(log,"Start Timestamp", False)
    df=DetectWeaknesses(logSorted, blacklist, maxTime,df,cols)
    df=Parallelizable_tasks_CaseLevel(df,cols)
    df.to_excel("Integrated_dataframe.xlsx")
    #print(df)
main()



Welcome to Joint Master thesis:
Modelling of production expertise to extend the data-driven analysis of process models


/Users/deepak/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecatedWarning: apply is deprecated as of 1.3.0 and will be removed in 2.0.0. Use algorithm entrypoint instead
  from ipykernel import kernelapp as app


Parallelizable_tasks_CaseLevel function




NameError: name 'ltl' is not defined